# 初见网络爬虫

本章将首先向网络服务器发送GET 请求以获取具体网页，再从网页中读取HTML 内容，
最后做一些简单的信息提取，将我们要寻找的内容分离出来。

1.0 向网络服务器发送GET请求以获取具体网页

2.0 从网页中读取HTML内容

3.0 信息提取、分离寻找的内容

### 1.1 网络链接

In [7]:
# Python实现方式

from urllib.request import urlopen
html = urlopen("http://pythonscraping.com/pages/page1.html")
print(html.read())

b'<html>\n<head>\n<title>A Useful Page</title>\n</head>\n<body>\n<h1>An Interesting Title</h1>\n<div>\nLorem ipsum dolor sit amet, consectetur adipisicing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.\n</div>\n</body>\n</html>\n'


In [3]:
%run scrapetest

b'<html>\n<head>\n<title>A Useful Page</title>\n</head>\n<body>\n<h1>An Interesting Title</h1>\n<div>\nLorem ipsum dolor sit amet, consectetur adipisicing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.\n</div>\n</body>\n</html>\n'


这将会输出http://pythonscraping.com/pages/page1.html 这个网页的全部HTML 代码。更
准确地说，这会输出在域名为http://pythonscraping.com 的服务器上< 网络应用根地址>/
pages 文件夹里的HTML 文件page1.html 的源代码。

**urllib 是Python 的标准库（就是说你不用额外安装就可以运行这个例子），包含了从网
络请求数据，处理cookie，甚至改变像请求头和用户代理这些元数据的函数。我们将在
本书中广泛使用urllib，所以建议你读读这个库的Python 文档（https://docs.python.org/3/
library/urllib.html）。**


**urlopen 用来打开并读取一个从网络获取的远程对象。因为它是一个非常通用的库（它可
以轻松读取HTML 文件、图像文件，或其他任何文件流），所以我们将在本书中频繁地使
用它。**

### 1.2 BeautifulSoup简介

BeautifulSoup 尝试化平淡为神奇。它通过定位HTML 标签来
格式化和组织复杂的网络信息，用简单易用的Python 对象为我们展现XML 结构信息。

BeautifulSoup 库最常用的对象恰好就是BeautifulSoup 对象。

In [16]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
html = urlopen("http://pythonscraping.com/pages/page1.html")
bsObj = BeautifulSoup(html.read(),'html.parser')#  BeautifulSoup(YOUR_MARKUP, "lxml")  markup_type=markup_type))
print(bsObj.h1)
print(bsObj.html.h1)
print(bsObj.html.body.h1)

<h1>An Interesting Title</h1>
<h1>An Interesting Title</h1>
<h1>An Interesting Title</h1>


我们导入urlopen，然后调用html.read() 获取网页的HTML 内容。这
样就可以把HTML 内容传到BeautifulSoup 对象，转换成下面的结构：

```html

html → <html><head>...</head><body>...</body></html>
— head → <head><title>A Useful Page<title></head>
— title → <title>A Useful Page</title>
— body → <body><h1>An Int...</h1><div>Lorem ip...</div></body>
— h1 → <h1>An Interesting Title</h1>
— div → <div>Lorem Ipsum dolor...</div>
```


我们从网页中提取的``<h1> ``标签被嵌在BeautifulSoup 对象bsObj 结构的第二层
（html → body → h1）。但是，当我们从对象里提取h1 标签的时候，可以直接调用它

```
bsObj.h1
```

其实，下面的所有函数调用都可以产生同样的结果：
```html
bsObj.html.body.h1
bsObj.body.h1
bsObj.html.h1.
```


**在第3 章，我们将进一步探讨一些更复杂的BeautifulSoup 函数，还会介绍正则表达式，以
及如何把正则表达式用于BeautifulSoup 以对网站信息进行提取**

### 1.3 可靠的网络链接

为什么一开始不估计可能会出现的异常！

让我们看看爬虫import 语句后面的第一行代码，如何处理那里可能出现的异常：

``html = urlopen("http://www.pythonscraping.com/pages/page1.html")``

这行代码主要可能会发生两种异常：
- 网页在服务器上不存在（或者获取页面的时候出现错误）
- 服务器不存在

#### 第一种异常发生时，程序会返回HTTP 错误

HTTP 错误可能是“404 Page Not Found”“500 Internal Server Error”等。所有类似情形，urlopen 函数都会抛出“HTTPError”异常。我们
可以用下面的方式处理这种异常:
```python
try:
html = urlopen("http://www.pythonscraping.com/pages/page1.html")
except HTTPError as e:
print(e)
# 返回空值，中断程序，或者执行另一个方案
else:
# 程序继续。注意：如果你已经在上面异常捕捉那一段代码里返回或中断（break），
# 那么就不需要使用else语句了，这段代码也不会执行
```

如果程序返回HTTP 错误代码，程序就会显示错误内容，不再执行else 语句后面的代码。

#### 如果服务器不存在（``就是说链接http://www.pythonscraping.com/ 打不开，或者是URL 链接写错了``），urlopen 会返回一个None 对象。

这个对象与其他编程语言中的null 类似。我们可以增加一个判断语句检测返回的html 是不是None：
```python
if html is None:
print("URL is not found")
else:
# 程序继续
```

当然，即使网页已经从服务器成功获取，如果网页上的内容并非完全是我们期望的那样，
仍然可能会出现异常.**每当你调用BeautifulSoup 对象里的一个标签时，增加一个检查条件
保证标签确实存在是很聪明的做法**。如果你想要调用的标签不存在，``BeautifulSoup`` 就会返回``None ``对象。不过，如果再调用这个None 对象下面的子标签，就会发生``AttributeError``错误。

下面这行代码（nonExistentTag 是虚拟的标签，BeautifulSoup 对象里实际没有）:
```python
print(bsObj.nonExistentTag)
```
会返回一个None 对象。处理和检查这个对象是十分必要的。如果你不检查，直接调用这个
None 对象的子标签，麻烦就来了。如下所示。
```python
print(bsObj.nonExistentTag.someTag)
```

这时就会返回一个异常：

```
AttributeError: 'NoneType' object has no attribute 'someTag'
```

**那么我们怎么才能避免这两种情形的异常呢？最简单的方式就是对两种情形进行检查：**

```python
try:
    badContent = bsObj.nonExistingTag.anotherTag   
except AttributeError as e:
    print("Tag was not found")
else:
    if badContent == None:
        print ("Tag was not found")
    else:
        print(badContent)
    
```

初看这些检查与错误处理的代码会觉得有点儿累赘，但是，我们可以重新简单组织一下代
码，让它变得不那么难写（更重要的是，不那么难读）。例如，下面的代码是上面爬虫的
另一种写法：

In [27]:
from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup


def getTitle(url):
    try:
        html = urlopen(url)
    except HTTPError as e:
        return None
    try:
        bsObj = BeautifulSoup(html.read(), "lxml")
        title = bsObj.body.h1
    except AttributeError as e:
        return None
    return title


title = getTitle("http://www.pythonscraping.com/pages/page1.html")
if title == None:
    print("Title could not be found")
else:
    print(title)
    

<h1>An Interesting Title</h1>


在这个例子中，我们创建了一个getTitle 函数，可以返回网页的标题，如果获取网页
的时候遇到问题就返回一个None 对象。在getTitle 函数里面，我们像前面那样检查了
HTTPError，然后把两行BeautifulSoup 代码封装在一个try 语句里面。这两行中的任何一
行有问题，AttributeError 都可能被抛出（如果服务器不存在，html 就是一个None 对象，
html.read() 就会抛出AttributeError）。其实，我们可以在try 语句里面放任意多行代码，
或者放一个在任意位置都可以抛出AttributeError 的函数。

在写爬虫的时候，思考代码的总体格局，让代码既可以捕捉异常又容易阅读，这是很重要
的。如果你还希望能够很大程度地重用代码，那么拥有像getSiteHTML 和getTitle 这样的
通用函数（具有周密的异常处理功能）会让快速稳定地网络数据采集变得简单易行。